In [109]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import math
from scipy import stats
import scanpy as sc
import xlsxwriter

In [110]:
#import data
global_dir_path = "C://Users/chris/OneDrive/Desktop/MIT postdoc project/ABN project/Lung cancer/Headspace_data"
#4wks
# dir_path = global_dir_path + '/2023_0809_Breath_EA_4w/Spectra/Peaks/Int_ratio'
# dir_path = global_dir_path + '/2023_1019_Breath_EA_4w/Spectra/Peaks/Int_ratio'
# #5wks
# dir_path = global_dir_path + '/2024_0520_Breath_EA_5w/Spectra/Peaks/Int_ratio'
# dir_path = global_dir_path + '/2024_1105_Breath_EA_5w/Spectra/Peaks/Int_ratio'
# #6wks
# dir_path = global_dir_path + '/2024_0116_Breath_EA_6w/Spectra/Peaks/Int_ratio' 
dir_path = global_dir_path + '/2024_0628_Breath_EA_6w/Spectra/Peaks/Int_ratio'

dir_list = sorted(os.listdir(dir_path))

In [111]:
df_dict = {}
for i in range(len(dir_list)):
    name = dir_path + '/' + dir_list[i]
    with open(name, mode="r", encoding="iso-8859-1") as file:
        df = pd.read_excel(name, index_col=0)
        
        df_dict[str(i)] = df

In [113]:
workbook = dir_path + '/matrix.xlsx'
writer = pd.ExcelWriter(workbook, engine = 'xlsxwriter')

# Write each DataFrame to a separate sheet in the Excel file
for sheet_name, df in df_dict.items():
    df.to_excel(writer, sheet_name=sheet_name, index=False)

writer.close()

In [114]:
xl = pd.ExcelFile(workbook)
sheets = xl.sheet_names
df1 = pd.concat(pd.read_excel(workbook, sheet_name=s).assign(Sample_ID = s) for s in sheets)

In [115]:
# df1.head(800)

,m/z,Intensity_ratio,Intensity,Sample_ID
0,27.0,0.044443,326.160307,0
1,31.0,0.009818,72.056753,0
2,32.9,0.001896,13.912569,0
3,33.0,1.736663,12745.224986,0
4,33.5,0.002459,18.049819,0
...,...,...,...,...
209,179.0,0.001849,17.078870,3
210,181.0,0.030649,283.026829,3
211,182.0,0.003375,31.163180,3
212,182.1,0.002401,22.167990,3


In [116]:
masses = [df1['m/z'].iloc[i] for i in range(len(df1))]

In [117]:
#set initial values of each mass as zero
d = {}
num_samples = len(dir_list) 
for mass in masses:
    d[mass] = [0]*num_samples

In [119]:
#add the actual values for samples with that m/z; if sample did not have that m/z, keep it zero
for i in range(len(df1)):
    mass = df1['m/z'].iloc[i]
    intensity = df1['Intensity_ratio'].iloc[i] 
    index = df1['Sample_ID'].iloc[i]
    d[mass][int(index)] = intensity

In [120]:
df2 = pd.DataFrame(data=d)

In [121]:
df2 = df2[sorted(df2.columns)]
df2

,26.9,27.0,31.0,31.1,32.9,33.0,33.5,33.6,33.8,34.0,...,251.0,251.1,269.9,279.0,281.0,281.1,281.2,282.1,283.1,297.1
0,0.000000,0.044443,0.009818,0.000000,0.001896,1.736663,0.002459,0.001365,0.001365,0.019612,...,0.000000,0.000000,0.000000,0.000000,0.004384,0.000000,0.000000,0.001365,0.001638,0.003560
1,0.000000,0.040143,0.011520,0.000000,0.000000,0.624178,0.000000,0.000000,0.000000,0.008291,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.052544,0.010360,0.000000,0.000000,0.838548,0.000000,0.000000,0.000000,0.009379,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.002195,0.002196,0.000000,0.000000,0.001916
3,0.000000,0.035833,0.009120,0.000000,0.000000,0.336083,0.000000,0.000000,0.000000,0.005327,...,0.000000,0.001193,0.000000,0.000000,0.001738,0.001957,0.000000,0.000000,0.000000,0.000000
4,0.001548,0.036020,0.007585,0.000000,0.000000,0.486564,0.000000,0.000000,0.000000,0.006809,...,0.000000,0.001519,0.000000,0.002025,0.000000,0.001927,0.000000,0.000000,0.000000,0.000000
5,0.000000,0.044306,0.007753,0.001244,0.001127,0.614338,0.000000,0.000000,0.000000,0.007052,...,0.001241,0.000000,0.000000,0.000000,0.002262,0.002152,0.000000,0.000000,0.000000,0.000000
6,0.000000,0.041670,0.010209,0.000000,0.000000,0.604773,0.000000,0.000000,0.000000,0.007838,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.001159,0.000000,0.000000,0.000000,0.000000
7,0.000000,0.037821,0.007399,0.001271,0.000000,0.458639,0.000000,0.000000,0.000000,0.005578,...,0.001046,0.000000,0.001046,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,0.000000,0.033855,0.009424,0.000000,0.000000,0.796739,0.000000,0.000000,0.000000,0.010614,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,0.000000,0.046707,0.012581,0.000000,0.000000,0.636439,0.000000,0.000000,0.000000,0.007468,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [122]:
df2.insert(0, "Sample_ID", dir_list)

In [123]:
types = ['Healthy' if '_C' in i or '_H' in i else 'Cancer' for i in df2['Sample_ID']]
df2.insert(0, 'Sample_type', types)

In [124]:
df2.to_excel(dir_path + '/dataframe.xlsx')

In [125]:
#perform zscore for each column
d2 = {}
num_samples = len(df2)
for col in df2.columns:
    d2[col] = [0]*num_samples

for col in df2.columns: 
    if type(col) == int or type(col) == float: 
        val = []
        for i in range(len(df2)):
            val.append(df2[col].loc[df2.index[i]])
        
        arr = np.array(val)
        score = stats.zscore(arr)
                
        for i in range(len(score)):
            d2[col][i] = score[i]

df3 = pd.DataFrame(data=d2)
df3 = df3.drop(['Sample_ID'], axis=1)
df3['Sample_type'] = types
df3.to_excel(dir_path + '/zscore.xlsx')

11


In [126]:
df3

,Sample_type,26.9,27.0,31.0,31.1,32.9,33.0,33.5,33.6,33.8,...,251.0,251.1,269.9,279.0,281.0,281.1,281.2,282.1,283.1,297.1
0,Cancer,-0.316228,0.629363,0.183750,-0.471373,2.676900,2.923398,3.162278,3.162278,3.162278,...,-0.469329,-0.467313,-0.316228,-0.316228,2.623482,-0.881350,-0.316228,3.162278,3.162278,2.751868
1,Cancer,-0.316228,-0.185510,1.283135,-0.471373,-0.453809,-0.215813,-0.316228,-0.316228,-0.316228,...,-0.469329,-0.467313,-0.316228,-0.316228,-0.552052,-0.881350,-0.316228,-0.316228,-0.316228,-0.447425
2,Cancer,-0.316228,2.164627,0.533683,-0.471373,-0.453809,0.389097,-0.316228,-0.316228,-0.316228,...,-0.469329,-0.467313,-0.316228,-0.316228,-0.552052,1.384671,3.162278,-0.316228,-0.316228,1.274958
3,Cancer,-0.316228,-1.002192,-0.267280,-0.471373,-0.453809,-1.028759,-0.316228,-0.316228,-0.316228,...,-0.469329,1.794606,-0.316228,-0.316228,0.706815,1.139327,-0.316228,-0.316228,-0.316228,-0.447425
4,Cancer,3.162278,-0.966674,-1.259634,-0.471373,-0.453809,-0.604131,-0.316228,-0.316228,-0.316228,...,-0.469329,2.411214,-0.316228,3.162278,-0.552052,1.107967,-0.316228,-0.316228,-0.316228,-0.447425
5,Cancer,-0.316228,0.603496,-1.151273,2.094050,1.407381,-0.243577,-0.316228,-0.316228,-0.316228,...,2.331826,-0.467313,-0.316228,-0.316228,1.086121,1.340404,-0.316228,-0.316228,-0.316228,-0.447425
6,Healthy,-0.316228,0.103996,0.436345,-0.471373,-0.453809,-0.270568,-0.316228,-0.316228,-0.316228,...,-0.469329,-0.467313,-0.316228,-0.316228,-0.552052,0.315729,-0.316228,-0.316228,-0.316228,-0.447425
7,Healthy,-0.316228,-0.625428,-1.379784,2.148307,-0.453809,-0.682931,-0.316228,-0.316228,-0.316228,...,1.892132,-0.467313,3.162278,-0.316228,-0.552052,-0.881350,-0.316228,-0.316228,-0.316228,-0.447425
8,Healthy,-0.316228,-1.376976,-0.070995,-0.471373,-0.453809,0.271120,-0.316228,-0.316228,-0.316228,...,-0.469329,-0.467313,-0.316228,-0.316228,-0.552052,-0.881350,-0.316228,-0.316228,-0.316228,-0.447425
9,Healthy,-0.316228,1.058429,1.968674,-0.471373,-0.453809,-0.181213,-0.316228,-0.316228,-0.316228,...,-0.469329,-0.467313,-0.316228,-0.316228,-0.552052,-0.881350,-0.316228,-0.316228,-0.316228,-0.447425
